In [1]:
#efektywność kodowania
#Eff = H / L
#gdzie:
# H - entropia 
# L - średnia długość słów kodowych
#Eff <0, 1>


In [2]:
from random import randint
import random as random
from math import log
import bitarray as ba
import operator

In [3]:
def read_file(file):
    return open(file).read()

In [4]:
def entropia(probability):
    h = 0
    for p in probability:
        h += p * log(p, 2)
    
    h = -h
    return h

In [5]:
def countChars(text2):
    chars = {}
    for char in list(text2):
        if char in chars:
            chars[char] += 1
        else:
            chars[char] = 1
    return chars

In [6]:
def obliczPrawdopodobienstwoWystapieniaNgramu(ngrams):
    probability = []
    occurences_sum = sum(ngrams.values())
    
    for ngram, occurences in ngrams.items():
        probability.append(occurences / occurences_sum)
        
    return probability

In [7]:
def obliczEntropieZnakowa(plik):
    counts = countChars(plik)
    probability = obliczPrawdopodobienstwoWystapieniaNgramu(counts)
    
    return entropia(probability)

In [8]:
wikipedia = read_file("norm_wiki_sample.txt")

In [9]:
entropia_wikipedii = obliczEntropieZnakowa(wikipedia)

In [10]:
#drzewa
#na początek każda litera jest drzewem
#bierzemy 2 z najmniejszym, prawdopodobieństwem
#łączymy je w jedno drzewo 
#ZNAK(prawdopodobieństwo)
# A(1/3) B(1/3) C(1/9) D(1/9) E(1/9)
#(krok 1)
#   CD (prawdopodobieństwo się sumuje)
#  /  \
# C    D

#Łączymy aż zostanie jedno drzewo
#(krok 2)
#        CDE
#       /   \ 
#      CD    E
#     /  \ 
#    C    D

#         ABCDE
#      0/      \1 
#      AB      CDE
#   00/  \01 10/  \11 
#    A    B   CD   E
#         100/ \101
#           C   D


#Do lewych krawędzi przypisujemy 0, do prawych 1 (bierzemy to co już było 
# i oddajemy znak)
#jak coś jest kodem to nie jest prefixem innego kodu
#w tym przykładzie wyjdzie:
#A - 00
#B - 01
#C - 100
#D - 101
#E - 11
#odczytujemy po jednym znaku i sprwadzamy, czy istnieje taki kod, jak istnieje to
#na pewno jest to już cały kod, bo żaden kod nie jest prefixem innego


In [11]:
efektywnosc_kodowania = entropia_wikipedii / 6

In [12]:
#Efektywnosc kodowania z ostatnich zajęć
print(efektywnosc_kodowania * 100, "%")


71.3399374450261 %


In [13]:
chars = countChars(wikipedia)

In [14]:
#DRZEWO
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.parent = None
        self.char = None
        self.prob = None

In [15]:
# OBSŁUGA DRZEWA
#          root = Tree()
#          root.data = "root"
#          root.left = Tree()
#          root.left.data = "left"
#          root.right = Tree()
#          root.right.data = "right"

In [19]:
def getLeaf(tree, code):
    #Istnieje lewe poddrzewo
    if(tree.left != None):
        code += "0"
        return getLeaf(tree.left, code)
    #istnieje prawe poddrzewo
    if(tree.right != None):
        code += "1"
        return getLeaf(tree.right, code)
    #zwraca liść
    
    return tree, code

In [20]:
#Tworzy kod na podstawie listy częstości znaków
def create(chars):
    code = {}
    
    #Posortowanie tak, aby najpierw były te występujące najrzadziej 
    sorted_chars = sorted(chars.items(), key=operator.itemgetter(1), reverse=False)
    print(type(sorted_chars))
    
    #Utwrzenie posrtowanej według prawdopodobieństw listy drzew
    tree_list = []
    for char, prob in sorted_chars:
        tree = Tree()
        tree.char = char
        tree.prob = prob
        tree_list.append(tree)
        
    while(len(tree_list) > 1) :
        #Łączenie drzew
        left_element = tree_list[0]
        right_element = tree_list[1]
        
        #Utworzenie nowego drzewa
        tree = Tree()
        tree.char = left_element.char + right_element.char
        tree.prob = left_element.prob + right_element.prob
        tree.left = left_element
        tree.right = right_element
        
        #dodanie rodzica do starych drzew
        left_element.parent = tree
        right_element.parent = tree
        
        
        #Usunięcie elementów scalonych w jedno drzewo
        del tree_list[0]
        del tree_list[0]
        
        #dodanie nowego drzewa do listy
        tree_list.append(tree)       
        #ponowne posortowanie listy drzew
        tree_list = sorted(tree_list, key=lambda x: x.prob, reverse=False)
        
   
    tree = tree_list[0]
    
    while(tree.right != None):
        new_code = ""
        leaf, new_code = getLeaf(tree, new_code)
        if(len(leaf.char) == 1):
            code[leaf.char] = new_code
        if(leaf.parent.left != None):
            leaf.parent.left = None
        else:
            leaf.parent.right = None
        
        
    return code

In [21]:
code = create(chars)

<class 'list'>


In [23]:
code

{'e': '000',
 'm': '00100',
 'y': '001010',
 'k': '0010110',
 '4': '001011100',
 'x': '001011101',
 '5': '001011110',
 '3': '001011111',
 's': '0011',
 'w': '010000',
 'b': '010001',
 'c': '01001',
 'r': '0101',
 'o': '0110',
 'n': '0111',
 'i': '1000',
 'd': '10010',
 '2': '10011000',
 '9': '10011001',
 'v': '1001101',
 'g': '100111',
 't': '1010',
 'p': '101100',
 'f': '101101',
 'l': '10111',
 'a': '1100',
 'h': '11010',
 '8': '110110000',
 'j': '110110001',
 '0': '11011001',
 'q': '1101101000',
 'z': '1101101001',
 '6': '1101101010',
 '7': '1101101011',
 '1': '11011011',
 'u': '110111',
 ' ': '111'}

In [24]:
#Koduje tekst
def encode(text, code):
    
    coded_text = ba.bitarray()
    for char in text:
        binary = code[char]
        
        for b in binary:
            if b == '1':
                coded_text.append(True)
            else:
                coded_text.append(False)
    
    return coded_text

In [25]:
import struct

In [26]:
#Dekoduje tekst
def decode(coded_text, code):
    text = []
    codeSwapped = dict((v,k) for k,v in code.items())
    bits = ""
    
    for i in range(0, len(coded_text), 1):
        if(coded_text[i] == True):
            bits += "1"
        else:
            bits += "0"
        if bits in codeSwapped:
            text.append(codeSwapped[str(bits)])
            bits = ""
        
    return ''.join(text) 

In [27]:
def save_file(file_name, code):
    result = open(file_name, 'w')
    
    for text, num in code.items():
        result.write(text + ";" + str(num) + ";")
    
    result.close()
    return result

In [28]:
def save_file_bin(file_name, bit_array):
    with open(file_name, 'wb') as fh:
        bit_array.tofile(fh)
    return 1

In [29]:
def read_file_bin(file_name):
    bit_array = ba.bitarray()
    with open(file_name, 'rb') as fh:
        bit_array.fromfile(fh)
    return bit_array

In [30]:
def read_bitarray(text):
    array = ba.bitarray()
    print(str.encode(text))
    array.frombytes(str.encode(text))

    return array   

In [31]:
#Zapis kodu i zakodowanego tekstu do pliku
def save(text, code, file_name, file_name_code):
    
    save_file_bin(file_name, text)
    save_file(file_name_code, code)
    
    return 0

In [32]:
#Wczytuje kod oraz tekst
def load(file_name, code_file_name):
    
    code_file = read_file(code_file_name)
    split = code_file.split(";")
    code = {}
    for char, num in zip(split[0::2], split[1::2]):
        code[char] = num
        
    text = read_file_bin(file_name)
    return text, code

In [33]:
def compare_strings(original, new_string):
    for org, new in zip(original, new_string):
        if org != new:
            print(org, "!=", new)
            return "Teksty są różne"
    return "Teksty sa równe"

In [34]:
encoded = encode(wikipedia, code)

In [35]:
save(encoded, code, "zakodowany", "kod")

0

In [36]:
coded, code = load("zakodowany", "kod")

In [37]:
decoded = decode(coded, code)

In [38]:
compare_strings(wikipedia, decoded)

'Teksty sa równe'

In [40]:
length = 0
i = 0

for char, c in code.items():
    i += chars[char]
    length += len(c) * chars[char]
    

oczekiwana_dlugosc_kodu = length / i

In [41]:
oczekiwana_dlugosc_kodu

4.3090155002237935

In [42]:
efektywnosc_kodowania2 = entropia_wikipedii / oczekiwana_dlugosc_kodu

In [43]:
efektywnosc_kodowania2

0.9933582848516693